In [13]:
%pip install -U langchain-openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Setting up LLM

In [44]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Create a GPT-4o model
llm = ChatOpenAI(model='gpt-4o',temperature=0,api_key=os.getenv("OPENAI_API_KEY"))

## Tools for LLM

In [48]:
import subprocess
from datetime import datetime
from langchain.tools import tool

@tool
def github_documentation_commit(commit_message=""):
    """ Commits the current changes to the 'documentation' branch of the Github repository """
    branch_name = "documentation"
    commit_message_main = f"{commit_message} - updated {datetime.now().strftime('%d-%m-%Y %H:%M:%S')}"
    
    # Switchs to the "documentation" branch (or Creates it if it doesn't exist)
    subprocess.run(["git", "checkout", "-B", branch_name])
    
    # Add all changes
    subprocess.run(["git", "add", "."])
    
    # Commit with the message
    subprocess.run(["git", "commit", "-m", commit_message_main])
    
    # Push to the remote repository
    subprocess.run(["git", "push", "-u", "origin", branch_name])
    
    return "Committed to Github on branch 'documentation'"

@tool
def create_file_tool(file_contents):
    """This function creates or updates a markdown file with the contents provided as input."""
    
    # Remove any Markdown formatting if present
    if file_contents.startswith("```") and file_contents.endswith("```"):
        file_contents = file_contents.strip("```").strip()
    
    with open("documentation.md", "w") as f:
        f.write(file_contents)
    
    return "File updated"


tools = [github_documentation_commit, create_file_tool]

## Simple Agent to use the tools

In [49]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent_executor = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent_executor.invoke(
    {
        "input": "Write a basic code documentation in markdown format. Output ONLY THE MARKDOWN without wrapping it in ```. Commit changes to 'documentation' branch with an appropriate commit message."
    }
)



> Entering new AgentExecutor chain...
